In this version, the oracle can annotate the suicide attempt and suicide planning of one sample at one time.

The oracle should complete the same number of planning-based annotations and attempt-based annotations at one time.

The annotated data generated by the machine learning are saved to a file named `result_file_Name`. The oracle should check the results, after which all annotated data should be moved to file `April_Submissions.xlsx` to enlarge the training dataset.

The oracle can input `12321` to terminate the annotation when asked to input 0 or 1 during the annotation.

In [ ]:
pip install keras-tcn --no-dependencies

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Libraries in use
import numpy as np
import pandas as pd
import openpyxl
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold 
from tcn import TCN, tcn_full_summary





In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/ActiveLearningStigma/ActiveLearningStigma/'
# training dataset
file_Name = root + "Data for machine annotation.xlsx"
# test dataset
test_file_Name = root + "Data from 2019 to 2020.xlsx"
# result file
result_file_Name = 'AnnotatedResult.xlsx'


In [ ]:
unclean_Data = pd.read_excel(file_Name,engine='openpyxl')
unclean_Data

,Index,id,author,Date,score,num_comments,title,selftext,Outcomes - Self-Harm/Suicide Attempts,Outcomes - Ideation/Planning,Outcomes - Self-Harm/Suicide Attempts.1,Outcomes - Ideation/Planning.1
0,0.0,alyhxj,Msmrme,2019-01-31 23:54:00,1.0,0.0,How long does it take to die in the cold?,Right now outside it’s -6°F feels like -18°F h...,0.0,How long does it take to die in the cold? ...,0.0,1.0
1,1.0,alyglb,imadethistofindhelp,2019-01-31 23:49:22,1.0,3.0,Looking for advice,One of my close friends says that he struggles...,0.0,struggles with thoughts of suicide,0.0,1.0
2,2.0,alyeff,whoagordoo,2019-01-31 23:41:45,1.0,1.0,I got into it with my dad tonight and really d...,My mom isn’t in my life and a large part is du...,0.0,I feel like I need to go,0.0,1.0
3,3.0,alydob,kuma1112,2019-01-31 23:39:08,1.0,0.0,Im... sorry the helper is now going to be dead...,I'm sorry..,0.0,now going to be dead,0.0,1.0
4,4.0,alyc26,HiddenAI,2019-01-31 23:33:48,1.0,1.0,People think I am useless.,"Nobody finds me useful. Recently, a person cal...",0.0,"I want to die, preferably by gassing myself",0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1536,NaN,b7w5em,ed8907,2019-03-31 22:27:26,23.0,6.0,Angel of Death.,**Angel of Death**. I know you're here because...,NaN,NaN,0.0,1.0
1537,NaN,b7w5c7,Hyppocritamus,2019-03-31 22:27:13,0.0,20.0,I cheated on my wife.,"She wants a divorce, and I'm not going to begr...",NaN,NaN,0.0,1.0
1538,NaN,b7w4pn,Anon1947473993,2019-03-31 22:25:26,1.0,3.0,Am going to do it in a couple days.,What is the best pill method w/ alcohol? I hav...,NaN,NaN,0.0,1.0
1539,NaN,b7x12p,morticiabates,2019-03-31 23:59:10,18.0,5.0,My mother makes me want to kill myself,"I'm in college, I work 4 days a week, but bec...",NaN,NaN,0.0,1.0


In [ ]:
raw_data = unclean_Data
raw_data = shuffle(raw_data)
# import pandas as pd
# raw_data = pd.read_excel('./Data fo machine annotation.xlsx', index=False)
label = raw_data.loc[:, ['Outcomes - Self-Harm/Suicide Attempts.1', 'Outcomes - Ideation/Planning.1']]
raw_data = raw_data.loc[:, ['title', 'selftext'], ]


test_unclean_Data = pd.read_excel(test_file_Name,engine='openpyxl')
test_data = test_unclean_Data.loc[:, ['title', 'selftext'], ]
test_data

,title,selftext
0,How long does it take to die in the cold?,Right now outside it’s -6°F feels like -18°F h...
1,Looking for advice,One of my close friends says that he struggles...
2,I got into it with my dad tonight and really d...,My mom isn’t in my life and a large part is du...
3,Im... sorry the helper is now going to be dead...,I'm sorry..
4,People think I am useless.,"Nobody finds me useful. Recently, a person cal..."
...,...,...
477,Angel of Death.,**Angel of Death**. I know you're here because...
478,I cheated on my wife.,"She wants a divorce, and I'm not going to begr..."
479,Am going to do it in a couple days.,What is the best pill method w/ alcohol? I hav...
480,NaN,NaN


In [ ]:

new_columns = raw_data.columns.tolist()
new_columns.insert(2, 'content')
data = raw_data.reindex(columns=new_columns)
# print(data.loc[:, 'title'].isnull().sum().sum())
data.loc[:,'content'] = data.loc[:, 'title'] + ' ' + data.loc[:, 'selftext']
data.fillna(method = 'ffill', axis = 1, inplace=True)
data.drop(['title', 'selftext'], axis = 1, inplace=True)
# data = pd.concat([data,label],axis=1)

test_new_columns = test_data.columns.tolist()
test_new_columns.insert(2, 'content')
testdata = test_data.reindex(columns=test_new_columns)
# print(data.loc[:, 'title'].isnull().sum().sum())
testdata.loc[:,'content'] = testdata.loc[:, 'title'] + ' ' + testdata.loc[:, 'selftext']
testdata.fillna(method = 'ffill', axis = 1, inplace=True)
testdata.drop(['title', 'selftext'], axis = 1, inplace=True)
testdata = testdata[0:479]

In [ ]:
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import WordPunctTokenizer
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
NLTK: 3.7
Scikit-learn: 1.0.2
Pandas: 1.3.5
Numpy: 1.21.6


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
text_messages = data['content']
# print(text_messages)
# Replace numbers with 'numbr'
processed = text_messages.str.replace(r'\d+(\.\d+)?', 'numbr')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

test_text_messages = testdata['content']
# print(text_messages)
# Replace numbers with 'numbr'
test_processed = test_text_messages.str.replace(r'\d+(\.\d+)?', 'numbr')

# Replace whitespace between terms with a single space
test_processed = test_processed.str.replace(r'\s+', ' ')

# Remove punctuation
test_processed = test_processed.str.replace(r'[^\w\d\s]', ' ')

# Remove leading and trailing whitespace
test_processed = test_processed.str.replace(r'^\s+|\s+?$', '')
test_processed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: Th

0      How long does it take to die in the cold  Righ...
1      Looking for advice One of my close friends say...
2      I got into it with my dad tonight and really d...
3      Im    sorry the helper is now going to be dead...
4      People think I am useless  Nobody finds me use...
                             ...                        
474    I think a decent job would prevent my suicide ...
475    i ve been making some of my worst decisions I ...
476    Hah  my life is cursed  It s weird looking at ...
477    Angel of Death    Angel of Death    I know you...
478    I cheated on my wife  She wants a divorce  and...
Name: content, Length: 479, dtype: object

In [ ]:
# change words to lower case - Hello, HELLO, hello are all the same word
processed = processed.str.lower()
processed

test_processed = test_processed.str.lower()
test_processed

0      how long does it take to die in the cold  righ...
1      looking for advice one of my close friends say...
2      i got into it with my dad tonight and really d...
3      im    sorry the helper is now going to be dead...
4      people think i am useless  nobody finds me use...
                             ...                        
474    i think a decent job would prevent my suicide ...
475    i ve been making some of my worst decisions i ...
476    hah  my life is cursed  it s weird looking at ...
477    angel of death    angel of death    i know you...
478    i cheated on my wife  she wants a divorce  and...
Name: content, Length: 479, dtype: object

In [ ]:
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer

all_words = []
all_bitext = []
tk = WordPunctTokenizer() 

for message in processed:
    
    words = tk.tokenize(message)
    for w in words:
        all_words.append(w)
    for i in range(len(words)-1):
        all_bitext.append(words[i] + words[i+1])

all_words = nltk.FreqDist(all_words)
all_bitext = nltk.FreqDist(all_bitext)

In [ ]:
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))
bi_Words = all_bitext.most_common(100)
print('Most common bitext: {}'.format(bi_Words))

Number of words: 8534
Most common words: [('i', 20725), ('to', 9053), ('and', 7264), ('the', 5970), ('it', 5312), ('my', 5147), ('a', 4771), ('t', 3801), ('of', 3748), ('me', 3514), ('that', 3275), ('m', 3230), ('in', 2622), ('is', 2442), ('for', 2422)]
Most common bitext: [('im', 3219), ('dont', 1602), ('andi', 1486), ('ihave', 1180), ('idon', 1179), ('ive', 1166), ('its', 1120), ('wantto', 1072), ('ican', 1021), ('thati', 869), ('iwas', 839), ('buti', 827), ('cant', 816), ('iam', 760), ('iwant', 642), ('ifi', 610), ('ijust', 605), ('ifeel', 591), ('mylife', 573), ('tobe', 521), ('inthe', 520), ('todo', 519), ('feellike', 462), ('iknow', 456), ('ofmy', 446), ('wheni', 440), ('becausei', 432), ('inmy', 422), ('iti', 414), ('mei', 411), ('todie', 395), ('tknow', 393), ('goingto', 383), ('togo', 367), ('doit', 357), ('tothe', 338), ('killmyself', 323), ('havea', 322), ('ina', 317), ('ithink', 305), ('vebeen', 304), ('forthe', 303), ('id', 302), ('likei', 293), ('icould', 291), ('mnot', 2

In [ ]:
word_features = []
bitext_features = []
for i in all_words.most_common(2500):
    word_features.append(i[0])
for  i in all_bitext.most_common(300):
    bitext_features.append(i[0])

In [ ]:
attempts_label = label['Outcomes - Self-Harm/Suicide Attempts.1']
planning_label = label['Outcomes - Ideation/Planning.1']
print(attempts_label, planning_label)

1240    0.0
1146    0.0
313     0.0
856     1.0
538     0.0
       ... 
371     0.0
1252    0.0
1384    0.0
640     0.0
915     0.0
Name: Outcomes - Self-Harm/Suicide Attempts.1, Length: 1541, dtype: float64 1240    1.0
1146    1.0
313     0.0
856     1.0
538     0.0
       ... 
371     0.0
1252    1.0
1384    1.0
640     0.0
915     1.0
Name: Outcomes - Ideation/Planning.1, Length: 1541, dtype: float64


In [ ]:
lexiconData = pd.read_excel('/content/drive/MyDrive/ActiveLearningStigma/ActiveLearningStigma/lexicon_Bi_Words.xlsx',engine='openpyxl')

In [ ]:
lexicon = [] 
for word in lexiconData['Words']:
  splitWord = word.split(',')
  lexicon.append(tuple(splitWord))
lexicon

[('want', 'die'),
 ('get', 'better'),
 ('want', 'live'),
 ('commit', 'suicid'),
 ('want', 'kill'),
 ('want', 'end'),
 ('live', 'life'),
 ('need', 'help'),
 ('suicid', 'thought'),
 ('self', 'harm'),
 ('end', 'life'),
 ('get', 'wors'),
 ('die', 'want'),
 ('tri', 'kill'),
 ('mental', 'ill'),
 ('suicid', 'attempt'),
 ('pleas', 'help'),
 ('get', 'help'),
 ('like', 'shit'),
 ('feel', 'better'),
 ('attempt', 'suicid'),
 ('life', 'want'),
 ('good', 'enough'),
 ('peopl', 'care'),
 ('wanna', 'die'),
 ('kill', 'self'),
 ('go', 'kill'),
 ('need', 'someon'),
 ('take', 'anymor'),
 ('suicid', 'note')]

In [ ]:
metk= MWETokenizer(lexicon)

In [ ]:
train_content = processed
train_attempts_label = attempts_label
train_planning_label = planning_label

def find_features(message):
    bitexts = []
    words = metk.tokenize(tk.tokenize(message))
    for i in range(len(words)-1):
        bitexts.append(words[i] + words[i+1])
    features = {}
    for bitext in bitext_features:
        features[bitext] = (bitext in bitexts)
    for word in word_features:
        features[word] = (word in words)
    return features

In [ ]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [ ]:
#load glove embedding
embeddings_dict = {}
with open("/content/drive/MyDrive/ActiveLearningStigma/ActiveLearningStigma/glove.42B.300d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

FileNotFoundError: ignored

In [ ]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [ ]:
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

#Tokenize 
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_content)

word_index = tokenizer.word_index
training_words_in_word2vector(embeddings_dict, word_index)




In [ ]:
sum=[]
def get_mean_vector(embeddings_dict):
    for word in embeddings_dict:
      sum.append(np.mean(embeddings_dict[word]))
    return (np.mean(sum), np.std(sum))

In [ ]:
emb_std, emb_mean = get_mean_vector(embeddings_dict)

In [ ]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
   
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map['handsome'].shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map[word]
            
    return embed_matrix

In [ ]:
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(embeddings_dict, w_2_i, emb_mean, emb_std)
em_matrix

In [ ]:
def tcn_model(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [ ]:
model_0 = tcn_model( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [ ]:
# Parameter Initialization
# Attempts 
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
record2=pd.DataFrame()
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8] #Modify this to faster training 

columns = ['Activation', 'Filters', 'acc1', 'acc2','acc3','acc4','acc5','AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 3 splits and shuffle = True
kfold = KFold(5, shuffle = True)

# Separate the sentences and the labels
sentences, labels, labels_2 = processed, attempts_label, planning_label
exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            train_y_2 ,test_y_2 = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])
                train_y_2.append(labels_2[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])
                test_y_2.append(labels_2[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)
            train_y_2 = np.array(train_y_2)
            test_y_2 = np.array(test_y_2)
          

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)
            testing_sequences = tokenizer.texts_to_sequences(test_processed)

            max_len1 = max_length(training_sequences)
            max_len2 = max_length(test_sequences)
            max_len3 = max_length(testing_sequences)
            max_len=max(max_len1,max_len2,max_len3)
            

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            testing = pad_sequences(testing_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            print(Xtrain.shape)
            print(Xtest.shape)
            print(testing.shape)


            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(embeddings_dict, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = tcn_model(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1,callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)

In [ ]:
print()
print(record2)
print()

In [ ]:
pred_val_y = model.predict([Xtrain])
pred_test_y = model.predict([Xtest])

In [ ]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(y_test, pred_test_y)

In [ ]:
from sklearn.metrics import f1_score
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    dataPlot=[]
    for threshold in [i * 0.01 for i in range(100)]:
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        dataPlot.append((threshold,score))
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    print(dataPlot)
    return (search_result, dataPlot)


In [ ]:
thresh,dataPlot  = threshold_search(train_y, pred_val_y)

In [ ]:
thresh['threshold']

In [ ]:
F1_score=[]
for data in dataPlot:
  F1_score.append(data[1])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(F1_score)

plt.show()

In [ ]:
plt.plot(dataPlot)

plt.show()

In [ ]:
Results=model.predict(testing)

In [ ]:
middle_threshold_result = []
found_threshold_result = []
for result in Results:
  if result >= .50:
    middle_threshold_result.append(1)
  else:
    middle_threshold_result.append(0)
  if result >= thresh['threshold']:
    found_threshold_result.append(1)
  else:
    found_threshold_result.append(0)

In [ ]:
Data = pd.read_excel(test_file_Name)
Data = Data.loc[:478,:"selftext"]
Data

In [ ]:
Data['attempts_middle_threshold_result'] = middle_threshold_result
Data['attempts_found_threshold_result'] = found_threshold_result

In [ ]:
# Parameter Initialization
# Attempts 
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
record2=pd.DataFrame()
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8] #Modify this to faster training 

columns = ['Activation', 'Filters', 'acc1', 'acc2','acc3','acc4','acc5','AVG']
record3 = pd.DataFrame(columns = columns)

exp = 0
acc_list = [] 
for activation in activations:
    for kernel_size in kernel_sizes:
        acc_list = [] 
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        for train, test in kfold.split(sentences):
          emb_matrix = pretrained_embedding_matrix(embeddings_dict, word_index, emb_mean, emb_std)
            
          # Define the input shape
          model_2 = tcn_model(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

          # Train the model
          model_2.fit(Xtrain, train_y_2, batch_size=50, epochs=100, verbose=1,callbacks=[callbacks], validation_data=(Xtest, test_y_2))

          # evaluate the model
          loss, acc = model_2.evaluate(Xtest, test_y_2, verbose=0)
          print('Test Accuracy: {}'.format(acc*100))
          acc_list.append(acc*100)
           
    mean_acc = np.array(acc_list).mean()
    parameters = [activation, kernel_size]
    entries = parameters + acc_list + [mean_acc]

    temp_2 = pd.DataFrame([entries], columns=columns)
    record2 = record2.append(temp_2, ignore_index=True)

In [ ]:
print()
print(record2)
print()

In [ ]:
pred_val_y = model_2.predict([Xtrain])
pred_test_y = model_2.predict([Xtest])

In [ ]:
thresh,dataPlot  = threshold_search(train_y, pred_val_y)

In [ ]:
thresh

In [ ]:
F1_score=[]
for data in dataPlot:
  F1_score.append(data[1])

In [ ]:
plt.plot(dataPlot)

plt.show()

In [ ]:
Results=model_2.predict(testing)

In [ ]:
middle_threshold_result = []
found_threshold_result = []
for result in Results:
  if result >= .50:
    middle_threshold_result.append(1)
  else:
    middle_threshold_result.append(0)
  if result >= thresh['threshold']:
    found_threshold_result.append(1)
  else:
    found_threshold_result.append(0)

In [ ]:
Data['Planning_middle_threshold_result'] = middle_threshold_result
Data['Planning_found_threshold_result'] = found_threshold_result

In [ ]:
from operator import index
Data.to_excel('/content/drive/MyDrive/ActiveLearningStigma/New_result_file.xlsx',index=False)